In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from models import ChopinNet
from models import BachNet
from utils import display_utils
from utils import graph_utils
from utils import prediction_utils
from utils import preprocessing_utils
from random import randint

import cv2
import gc
import keras
import os
import numpy as np
import networkx as nx
import re
import shutil
import sys

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

Using TensorFlow backend.
/home/collin/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
train_path = "data/train/input/"
test_path = "data/test/input/"

output_path = "output"

path_to_original_images = "original/"
path_to_gt_images = "gt/"

input_image_shape = (100, 100)
receptive_field_shape = (23, 23)

In [3]:
def train_single(chopin,
                 img,
                 I_a,
                 gt,
                 seeds,
                 foldername,
                 global_loss=[],
                 global_accuracy=[],
                 num_epochs=8):
    
    
    I_a = preprocessing_utils.pad_for_window(I_a,
                               chopin.receptive_field_shape[0],
                               chopin.receptive_field_shape[1])
    
    graph = graph_utils.prims_initialize(img)

    ground_truth_cuts, gt_assignments = graph_utils.generate_gt_cuts(gt,
                                                                     seeds,
                                                                     assignments=True)

    local_loss = []
    local_accuracy = []
    
    os.mkdir(os.path.join(foldername, "saved_models"))

    for epoch in range(num_epochs):
        print("Epoch {}".format(epoch + 1))
        msf = prediction_utils.minimum_spanning_forest(chopin, I_a, graph, seeds)

        segmentations = display_utils.assignments(img, msf, seeds)

        shortest_paths = nx.get_node_attributes(msf, 'path')
        assignments = nx.get_node_attributes(msf, 'seed')
        cuts = graph_utils.get_cut_edges(msf)

        acc = graph_utils.accuracy(assignments, gt_assignments)
        
        print("Accuracy: ", acc)
        local_accuracy.append(acc)

        filename = "epoch_{}.png".format(epoch + 1)
        
        boundaries = display_utils.view_boundaries(np.zeros_like(img),
                                                   cuts)

        plt.imsave(os.path.join(foldername, filename), boundaries)

        constrained_msf = msf.copy()

        constrained_msf.remove_edges_from(ground_truth_cuts)

        constrained_msf = graph_utils.minimum_spanning_forest(constrained_msf,
                                                        seeds)

        ground_truth_paths = nx.get_node_attributes(constrained_msf, 'path')

        children = graph_utils.compute_root_error_edge_children(shortest_paths,
                                                          ground_truth_paths,
                                                          cuts,
                                                          ground_truth_cuts)

        weights = []
        static_images = []
        dynamic_images = []

        loss = 0

        for (u, v), weight in children.iteritems():
            weights.append(weight)
            static_images.append(msf.get_edge_data(u, v)['static_image'][0])
            dynamic_images.append(msf.get_edge_data(u, v)['dynamic_image'][0])
            altitude_val = msf.get_edge_data(u, v)['weight']

            loss += weight * altitude_val

        batches = zip(preprocessing_utils.create_batches(np.expand_dims(np.stack(weights), 1)),
                      preprocessing_utils.create_batches(np.stack(static_images)),
                      preprocessing_utils.create_batches(np.stack(dynamic_images)))

        with chopin.sess.as_default():
            chopin.sess.run(chopin.zero_ops)

            for w, s, d in batches:
                feed_dict = {chopin.gradient_weights: w.transpose(),
                             chopin.static_input: s,
                             chopin.dynamic_input: d,
                             keras.backend.learning_phase(): 0}

                chopin.sess.run(chopin.accum_ops, feed_dict)

            chopin.sess.run(chopin.train_step)



        local_loss.append(loss)
        print("Loss: ", loss)
        
        info = "Epoch: {}\tloss: {}\taccuracy: {}\n".format(epoch + 1, loss, acc)
        loss_file.write(info)
        loss_file.flush()

        f, axarr = plt.subplots(2, sharex=True)
        axarr[0].plot(local_loss)
        axarr[0].set_title("Loss")
        axarr[1].plot(local_accuracy)
        axarr[1].set_title("Accuracy")

        figurename = "Local Loss and Accuracy"

        plt.savefig(os.path.join(foldername, figurename))
        
        global_loss.append(loss)
        global_accuracy.append(acc)
        f, axarr = plt.subplots(2, sharex=True)
        axarr[0].plot(global_loss)
        axarr[0].set_title("Loss")
        axarr[1].plot(global_accuracy)
        axarr[1].set_title("Accuracy")

        figurename = "Global Loss and Accuracy"

        global_folder = foldername.split("/")[:-1]
        global_folder = "/".join(global_folder)
        
        plt.savefig(os.path.join(global_folder, figurename))
        

        chopin.save_model("models/saved_model/Chopin/model.ckpt")
        model_name = "epoch_{}".format(epoch)
        chopin.save_model(os.path.join(foldername, "saved_models", model_name, model_name))
    gc.collect()

    return segmentations, global_loss, global_accuracy

In [ ]:
new_size = (100, 100)

# Resize images

files = os.listdir(train_path)

for i, filename in enumerate(files):
    
    sys.stdout.write("\rProgress: %.2f%% || %d/%d" % (i / len(files),
                                                      i,
                                                      len(files)))
    sys.stdout.flush()
    
    f_name, ext = os.path.splitext(filename)
    
    if "gt" in f_name.split("_"):
        continue
        
    gt_filename = f_name + "_gt"
    
    gt_path = os.path.join(train_path, (gt_filename + ext))
        
    if not os.path.isfile(gt_path):
        continue    
    
    foldername = "data/train/chopin/" + f_name
        
    image_path = os.path.join(train_path, filename)
    
    img = cv2.imread(image_path, 0)
    gt = cv2.imread(gt_path, 0)
    
    tl_corner = (randint(0, (img.shape[0] - new_size[0])),
             randint(0, (img.shape[1] - new_size[1])))

    resized_img = preprocessing_utils.crop_2d(img, tl_corner, new_size[0], new_size[1])
    resized_gt = preprocessing_utils.crop_2d(gt, tl_corner, new_size[0], new_size[1])
    
    plt.imsave(image_path, resized_img, cmap='gray')
    plt.imsave(gt_path, resized_gt, cmap='gray')
    
    gc.collect()
    
sys.stdout.write("\rProgress: Done! || %d/%d" % (len(files),
                                                len(files)))
sys.stdout.flush()

Progress: Done! || 319/319

In [ ]:
# Import Images

bach = BachNet.BachNet()
bach.build(23, 23, 1)
bach.load_model('models/saved_model/Bach/model.h5')

image_data = dict()

files = os.listdir(train_path)

for i, filename in enumerate(files):
    
    sys.stdout.write("\rProgress: %.2f%% || %d/%d" % (i / len(files),
                                                      i,
                                                      len(files)))
    
    f_name, ext = os.path.splitext(filename)
    
    if "gt" in f_name.split("_"):
        continue
        
    gt_filename = f_name + "_gt"
    
    gt_path = os.path.join(train_path, (gt_filename + ext))
        
    if not os.path.isfile(gt_path):
        continue    
    
    foldername = "data/train/chopin/" + f_name
        
    image_path = os.path.join(train_path, filename)
    
    img, gt, seeds = preprocessing_utils.load_image(foldername,
                                    f_name,
                                    image_path,
                                    gt_path) 
    
    bps = prediction_utils.boundary_probabilities(bach, img)
    
    image_data[f_name] = img, bps, gt, seeds
    
sys.stdout.write("\rProgress: Done! || %d/%d" % (len(files),
                                                len(files)))
sys.stdout.flush()


chopin = ChopinNet.Chopin()
chopin.build(receptive_field_shape)
chopin.initialize_session()
chopin.load_model("models/saved_model/Chopin/model.ckpt")


global_loss = list()
global_accuracy = list()

try:
    loss_file
except NameError:
    loss_file = open('data/train/chopin/global_loss.txt', 'w')



for name, (img, bps, gt, seeds) in image_data.iteritems():
    print("Training on " + name)
    
    img_info = "\nImage: {}\n\n".format(name)
    loss_file.write(img_info)
    loss_file.flush()
    
    I_a = np.stack((img, bps), axis=2)
    foldername = "data/train/chopin/" + name
    segs, glob, acc_timeline = train_single(chopin,
                                            img,
                                            I_a,
                                            gt,
                                            seeds,
                                            foldername,
                                            global_loss,
                                            global_accuracy,
                                            num_epochs=16)
    
loss_file.close()

Progress: 0.00% || 0/319data/train/input/slice_78_gt.png data/train/chopin/slice_78
Progress: 0.02% || 7/319data/train/input/slice_70_gt.png data/train/chopin/slice_70
Progress: 0.03% || 8/319data/train/input/slice_23_gt data/train/chopin/slice_23
Progress: 0.03% || 9/319data/train/input/slice_63_gt data/train/chopin/slice_63
Progress: 0.04% || 14/319data/train/input/slice_59_gt.png data/train/chopin/slice_59
Progress: 0.05% || 16/319data/train/input/slice_67_gt data/train/chopin/slice_67
Progress: 0.05% || 17/319data/train/input/slice_55_gt data/train/chopin/slice_55
Progress: 0.06% || 18/319data/train/input/slice_46_gt data/train/chopin/slice_46
Progress: 0.06% || 20/319data/train/input/slice_71_gt.png data/train/chopin/slice_71
Progress: 0.07% || 21/319data/train/input/slice_47_gt.png data/train/chopin/slice_47
Progress: 0.09% || 29/319data/train/input/slice_85_gt data/train/chopin/slice_85
Progress: 0.10% || 32/319data/train/input/slice_5_gt.png data/train/chopin/slice_5
Progress: 